In [17]:
import psycopg2 as p
import getpass, time
from datetime import datetime

In [33]:
start_time = time.time()

In [34]:
password = 'Evergreen1'

conn = p.connect(database = 'postgres', user = 'postgres', password = password, 
                       host = 'localhost', port = 5433)

cur = conn.cursor()

In [35]:
# Rule V (General MCMIS Validation Criteria)
# Uses the MCMIS validation criteria to then insert into mcmiscandidate

cur.execute(
"""
with validation_criteria as (select * from mcmis
where 
    mcs150_date > now() - interval '2 years' and
    phy_natn = 'US' and mai_natn = 'US' and
    class is not null and
    class similar to ('G%|A%|B%|C%|L%') and (class not similar to ('_E%|_F%|_H%|_I%|_J%|_K%') and
    class not similar to ('D%|E%|F%|H%|I%|J%|K%')) and
    (carship in ('C', 'S') or carship like '%C%') and not
    (passengers is not null and not (cargoothr is null or othercargo is null or genfreight is null or 
                                     household is null or metalsheet is null or motorveh is null or 
                                     drivetow is null or logpole is null or bldgmat is null or 
                                     mobilehome is null or machlrg is null or produce is null or 
                                     oilfield is null or liqgas is null or intermodal is null or 
                                     livestock is null or grainfeed is null or coalcoke is null or 
                                     meat is null or garbage is null or usmail is null or chem is null or 
                                     drybulk is null or coldfood is null or beverages is null or 
                                     paperprod is null or utility is null or farmsupp is null or 
                                     construct is null or waterwell is null or cargoothr is null or 
                                     othercargo is null))
group by 
    mcmis_id, act_stat, carship, census_num, name, name_dba, dbnum, phy_natn, 
    reg, phy_str, phy_city, phy_cnty, phy_st, phy_zip, undeliv_phy, tel_num, 
    cell_num, fax_num, mai_natn, mai_str, mai_city, mai_cnty, mai_st, mai_zip, 
    undeliv_mai, oic, terr, iccdocket1, icc1, iccdocket2, icc2, iccdocket3, icc3, 
    class, classdef, crrinter, crrhmintra, crrintra, shpinter, shpintra, 
    vehicleregistrant, org, genfreight, household, metalsheet, motorveh, drivetow, 
    logpole, bldgmat, mobilehome, machlrg, produce, liqgas, intermodal, passengers, 
    oilfield, livestock, grainfeed, coalcoke, meat, garbage, usmail, chem, drybulk, 
    coldfood, beverages, paperprod, utility, farmsupp, construct, waterwell, 
    cargoothr, othercargo, hm_ind, owntruck, owntract, owntrail, owncoach, ownschool_1_8,
     ownschool_9_15, ownschool_16, ownbus_16, ownvan_1_8, ownvan_9_15, ownlimo_1_8, 
    ownlimo_9_15, ownlimo_16, trmtruck, trmtract, trmtrail, trmcoach, trmschool_1_8,
     trmschool_9_15, trmschool_16, trmbus_16, trmvan_1_8, trmvan_9_15, trmlimo_1_8,
     trmlimo_9_15, trmlimo_16, trptruck, trptract, trptrail, trpcoach, trpschool_1_8,
     trpschool_9_15, trpschool_16, trpbus_16, trpvan_1_8, trpvan_9_15, trplimo_1_8,
     trplimo_9_15, trplimo_16, tot_trucks, tot_buses, tot_pwr, fleetsize, interlt100,
     intergt100, inter_drs, intralt100, intragt100, intra_drs, avg_tld, tot_drs,
     cdl_drs, revtype, revdocnum, revdate, acc_rate, repprevrat, mlg150, miletot,
     rating, ratedate, phy_barrio, mai_barrio, mcsipstep, mcsipdate, userid, addcode,
     upd_reas, delcode, mcs150mileageyear, adddate, chngdate, deldate, tot_cars,
     version, createdate, adduserid, deluserid, mcs150_date, rec_updated_flag,
     email_address, usdot_revoked_flag, usdot_revoked_number, company_rep1, company_rep2

having sum(owntract+trmtract+trptract+owntruck+trmtruck+trptruck) > 0 and not
    (sum(owntruck+trmtruck+trptruck) = 1 and sum(owntract+trmtract+trptract) = 0 and sum(owntrail+trmtrail+trptrail) = 0)
    
)

insert into mcmiscandidate
select *
from validation_criteria as vc
where not exists (select * from fleetseek.fleet as b where vc.census_num = b.usdot)
on conflict (census_num) do nothing
"""
)
conn.commit()

## 1/17/2020 rewrite

In [36]:
# As of 1/17/2020 able to effectively update under a minute


cur.execute("""

update public.mcmiscandidate 
set reject_reason = 'Owner-Operator'
from (
    select *
    from public.mcmiscandidate
    where
        passengers is null and
        carship like 'C%' and
        (class similar to 'A%|B%' or
        class = 'AB') and 
        mcs150_date > now() - interval '2 years' and
        tot_buses = 0


    group by 
        mcmis_id, act_stat, carship, census_num, name, name_dba, dbnum, phy_natn, 
        reg, phy_str, phy_city, phy_cnty, phy_st, phy_zip, undeliv_phy, tel_num, 
        cell_num, fax_num, mai_natn, mai_str, mai_city, mai_cnty, mai_st, mai_zip, 
        undeliv_mai, oic, terr, iccdocket1, icc1, iccdocket2, icc2, iccdocket3, icc3, 
        class, classdef, crrinter, crrhmintra, crrintra, shpinter, shpintra, 
        vehicleregistrant, org, genfreight, household, metalsheet, motorveh, drivetow, 
        logpole, bldgmat, mobilehome, machlrg, produce, liqgas, intermodal, passengers, 
        oilfield, livestock, grainfeed, coalcoke, meat, garbage, usmail, chem, drybulk, 
        coldfood, beverages, paperprod, utility, farmsupp, construct, waterwell, 
        cargoothr, othercargo, hm_ind, owntruck, owntract, owntrail, owncoach, ownschool_1_8,
         ownschool_9_15, ownschool_16, ownbus_16, ownvan_1_8, ownvan_9_15, ownlimo_1_8, 
        ownlimo_9_15, ownlimo_16, trmtruck, trmtract, trmtrail, trmcoach, trmschool_1_8,
         trmschool_9_15, trmschool_16, trmbus_16, trmvan_1_8, trmvan_9_15, trmlimo_1_8,
         trmlimo_9_15, trmlimo_16, trptruck, trptract, trptrail, trpcoach, trpschool_1_8,
         trpschool_9_15, trpschool_16, trpbus_16, trpvan_1_8, trpvan_9_15, trplimo_1_8,
         trplimo_9_15, trplimo_16, tot_trucks, tot_buses, tot_pwr, fleetsize, interlt100,
         intergt100, inter_drs, intralt100, intragt100, intra_drs, avg_tld, tot_drs,
         cdl_drs, revtype, revdocnum, revdate, acc_rate, repprevrat, mlg150, miletot,
         rating, ratedate, phy_barrio, mai_barrio, mcsipstep, mcsipdate, userid, addcode,
         upd_reas, delcode, mcs150mileageyear, adddate, chngdate, deldate, tot_cars,
         version, createdate, adduserid, deluserid, mcs150_date, rec_updated_flag,
         email_address, usdot_revoked_flag, usdot_revoked_number, company_rep1, company_rep2,
         reject_reason, product_code, fleet_type_id, insertion_flag

    having 
        (sum(owntract+trmtract+trptract) = 1 and sum(owntrail+ trmtrail+ trptrail) <= 1) and 
        sum(owntruck+trmtruck+trptruck) = 0
        ) as sub_q
where mcmiscandidate.census_num = sub_q.census_num 

""")
conn.commit()

In [37]:
# Rule Y (For Hire criteria)
cur.execute(
"""
update mcmiscandidate
set reject_reason = 'For-Hire'
from (
    select *
    from mcmiscandidate
    where 
        class in ('A', 'AB', 'ABD', 'ABG', 'ABL', 'AG', 'AGL', 'AL', 'B', 'BL')

    group by
        mcmis_id, act_stat, carship, census_num, name, name_dba, dbnum, phy_natn, 
        reg, phy_str, phy_city, phy_cnty, phy_st, phy_zip, undeliv_phy, tel_num, 
        cell_num, fax_num, mai_natn, mai_str, mai_city, mai_cnty, mai_st, mai_zip, 
        undeliv_mai, oic, terr, iccdocket1, icc1, iccdocket2, icc2, iccdocket3, icc3, 
        class, classdef, crrinter, crrhmintra, crrintra, shpinter, shpintra, 
        vehicleregistrant, org, genfreight, household, metalsheet, motorveh, drivetow, 
        logpole, bldgmat, mobilehome, machlrg, produce, liqgas, intermodal, passengers, 
        oilfield, livestock, grainfeed, coalcoke, meat, garbage, usmail, chem, drybulk, 
        coldfood, beverages, paperprod, utility, farmsupp, construct, waterwell, 
        cargoothr, othercargo, hm_ind, owntruck, owntract, owntrail, owncoach, ownschool_1_8,
         ownschool_9_15, ownschool_16, ownbus_16, ownvan_1_8, ownvan_9_15, ownlimo_1_8, 
        ownlimo_9_15, ownlimo_16, trmtruck, trmtract, trmtrail, trmcoach, trmschool_1_8,
         trmschool_9_15, trmschool_16, trmbus_16, trmvan_1_8, trmvan_9_15, trmlimo_1_8,
         trmlimo_9_15, trmlimo_16, trptruck, trptract, trptrail, trpcoach, trpschool_1_8,
         trpschool_9_15, trpschool_16, trpbus_16, trpvan_1_8, trpvan_9_15, trplimo_1_8,
         trplimo_9_15, trplimo_16, tot_trucks, tot_buses, tot_pwr, fleetsize, interlt100,
         intergt100, inter_drs, intralt100, intragt100, intra_drs, avg_tld, tot_drs,
         cdl_drs, revtype, revdocnum, revdate, acc_rate, repprevrat, mlg150, miletot,
         rating, ratedate, phy_barrio, mai_barrio, mcsipstep, mcsipdate, userid, addcode,
         upd_reas, delcode, mcs150mileageyear, adddate, chngdate, deldate, tot_cars,
         version, createdate, adduserid, deluserid, mcs150_date, rec_updated_flag,
         email_address, usdot_revoked_flag, usdot_revoked_number, company_rep1, company_rep2,
         reject_reason, product_code, fleet_type_id, insertion_flag

    having 
        (sum(owntruck + trmtruck + trptruck) >= 2 or
        (sum(owntruck + trmtruck + trptruck) = 1 and sum(owntract + trmtract + trptract) = 1) or
        (sum(owntruck + trmtruck + trptruck) >= 1 and sum(owntract + trmtract + trptract) = 0 and sum(owntrail + trmtrail + trptrail) >= 2) or
        (sum(owntruck + trmtruck + trptruck) = 0 and sum(owntract + trmtract + trptract) >= 1 and sum(owntrail + trmtrail + trptrail) >= 2))
        ) as sub_q

where mcmiscandidate.census_num = sub_q.census_num;

"""    
)
conn.commit()

In [38]:
# Rule Z (Private criteria)

# 12/12/2019 Change classes and remove greater than 10 rule
cur.execute(
"""
update mcmiscandidate
set reject_reason = 'Private Fleet'
from (
    select * 
    from mcmiscandidate
    where
        mcs150_date > now() - interval '2 years'
    group by
        mcmis_id, act_stat, carship, census_num, name, name_dba, dbnum, phy_natn, 
            reg, phy_str, phy_city, phy_cnty, phy_st, phy_zip, undeliv_phy, tel_num, 
            cell_num, fax_num, mai_natn, mai_str, mai_city, mai_cnty, mai_st, mai_zip, 
            undeliv_mai, oic, terr, iccdocket1, icc1, iccdocket2, icc2, iccdocket3, icc3, 
            class, classdef, crrinter, crrhmintra, crrintra, shpinter, shpintra, 
            vehicleregistrant, org, genfreight, household, metalsheet, motorveh, drivetow, 
            logpole, bldgmat, mobilehome, machlrg, produce, liqgas, intermodal, passengers, 
            oilfield, livestock, grainfeed, coalcoke, meat, garbage, usmail, chem, drybulk, 
            coldfood, beverages, paperprod, utility, farmsupp, construct, waterwell, 
            cargoothr, othercargo, hm_ind, owntruck, owntract, owntrail, owncoach, ownschool_1_8,
             ownschool_9_15, ownschool_16, ownbus_16, ownvan_1_8, ownvan_9_15, ownlimo_1_8, 
            ownlimo_9_15, ownlimo_16, trmtruck, trmtract, trmtrail, trmcoach, trmschool_1_8,
             trmschool_9_15, trmschool_16, trmbus_16, trmvan_1_8, trmvan_9_15, trmlimo_1_8,
             trmlimo_9_15, trmlimo_16, trptruck, trptract, trptrail, trpcoach, trpschool_1_8,
             trpschool_9_15, trpschool_16, trpbus_16, trpvan_1_8, trpvan_9_15, trplimo_1_8,
             trplimo_9_15, trplimo_16, tot_trucks, tot_buses, tot_pwr, fleetsize, interlt100,
             intergt100, inter_drs, intralt100, intragt100, intra_drs, avg_tld, tot_drs,
             cdl_drs, revtype, revdocnum, revdate, acc_rate, repprevrat, mlg150, miletot,
             rating, ratedate, phy_barrio, mai_barrio, mcsipstep, mcsipdate, userid, addcode,
             upd_reas, delcode, mcs150mileageyear, adddate, chngdate, deldate, tot_cars,
             version, createdate, adduserid, deluserid, mcs150_date, rec_updated_flag,
             email_address, usdot_revoked_flag, usdot_revoked_number, company_rep1, company_rep2,
             reject_reason, product_code, fleet_type_id, insertion_flag

    having 
        (class in ('ABC', 'ABCD', 'ABCG', 'ABCL', 'ABGL', 'C%', 'AC', 'ACD', 'ACDL', 'ACG', 'ACGL', 'ACL', 'AD', 'BC', 'BCG', 'BCGL', 'BCL') and 
        sum(owntruck+trmtruck+trptruck+owntract+trmtract+trptract) = 1 and sum(owntrail+trmtrail+trptrail) >= 2 or 
        (class in ('ABC', 'ABCD', 'ABCG', 'ABCL', 'ABGL', 'C%', 'AC', 'ACD', 'ACDL', 'ACG', 'ACGL', 'ACL', 'AD', 'BC', 'BCG', 'BCGL', 'BCL') and 
        sum(owntruck+trmtruck+trptruck+owntract+trmtract+trptract) >= 2)) 
    ) as sub_q
where mcmiscandidate.census_num = sub_q.census_num;
"""
)
conn.commit()

In [39]:
#Rule BB (Other candidate criteria)

cur.execute(
"""
update mcmiscandidate
set reject_reason = 'Other candidate criteria'
where class like 'L%'
"""
)
conn.commit()

In [40]:
# Rule to set product codes and fleet_type_ids 

cur.execute("""
update mcmiscandidate 
set 
    product_code = 'PFD',
    fleet_type_id = 106
where reject_reason = 'Private Fleet';

update mcmiscandidate
set 
    product_code = 'NMC',
    fleet_type_id = 104
where reject_reason = 'For-Hire';

update mcmiscandidate
set
    product_code = 'OOD',
    fleet_type_id = 105
where reject_reason = 'Owner-Operator';

""")


conn.commit()
conn.close()

In [41]:
end_time = time.time() - start_time

In [42]:
print(end_time)

33.25920844078064
